### IMPORTS & DATASET

In [1]:
import os, cv2, keras, matplotlib.pyplot as plt, numpy as np, pandas as pd
from keras.layers import Dense
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from tqdm import tqdm

In [ ]:
!unzip '/content/Images.zip' -d '/content/images'
!unzip '/content/Airplanes_Annotations.zip' -d '/content/labels'

### PROCESSING FUNCTIONS

In [2]:
def get_iou(bb_gt, bb_pred):
    """
    This function computes the iou (intersection over union)

    PARAMS:
    bb_gt --> type: dict, contains labeled bounding boxes
    bb_pred --> type: dict, contains predicted bounding boxes

    RETURNS:
    iou --> type: int, the iou score (accuracy)
    """

    assert bb_gt['x1'] < bb_gt['x2']
    assert bb_gt['y1'] < bb_gt['y2']
    assert bb_pred['x1'] < bb_pred['x2']
    assert bb_pred['y1'] < bb_pred['y2']

    x_left = max(bb_gt['x1'], bb_pred['x1'])
    y_top = max(bb_gt['y1'], bb_pred['y1'])
    x_right = min(bb_gt['x2'], bb_pred['x2'])
    y_bottom = min(bb_gt['y2'], bb_pred['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb_gt_area = (bb_gt['x2'] - bb_gt['x1']) * (bb_gt['y2'] - bb_gt['y1'])
    bb_pred_area = (bb_pred['x2'] - bb_pred['x1']) * (bb_pred['y2'] - bb_pred['y1'])

    iou = intersection_area / float(bb_gt_area + bb_pred_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

# get intersection over union (iou)
# source (https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)

In [3]:
cv2.setUseOptimized(True)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

def create_training_set(image_dir, label_dir):
  assert(len(os.listdir(image_dir)) == len(os.listdir(label_dir))) ## make sure all images have a label

  X_train = []
  y_train = []
  
  for idx, file_ in enumerate(tqdm(os.listdir(label_dir), desc='PROCESSING IMAGES', unit='IMAGE')):
      if file_.startswith("airplane"):
          filename = file_.split(".")[0]+".jpg"
          image = cv2.imread(os.path.join(image_dir, filename)) ## read in image
          df = pd.read_csv(os.path.join(label_dir, file_)) ## read in labels

          gt_values=[] ## ground_truth bounding box labels
          for row in df.iterrows():
              x1 = int(row[1][0].split(" ")[0])
              y1 = int(row[1][0].split(" ")[1])
              x2 = int(row[1][0].split(" ")[2])
              y2 = int(row[1][0].split(" ")[3])
              gt_values.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})

          ss.setBaseImage(image)
          ss.switchToSelectiveSearchFast()
          ssresults = ss.process()
          imout = image.copy()

          counter = 0 ## collect at a maximum 30 positive samples from a single image
          falsecounter = 0 ## collect at a maximum 30 negative samples from a single image

          for idx, prediction in enumerate(ssresults): ## loop over all recognized objects in the image
              if idx < 2000:
                  for gtval in gt_values: ## loop over all ground_truth labels in this image
                      x,y,w,h = prediction
                      iou = get_iou(gtval, {"x1":x,"x2":x+w,"y1":y,"y2":y+h}) ## calculate our iou
                      if counter < 30: ## collect only 30 samples negative or positive
                          if iou > 0.70: ## if iou is above 70% --> we have identified an airplane
                              timage = imout[y:y+h,x:x+w] ## create a new image with only the specified coordinates of our object (airplane)
                              resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                              X_train.append(resized)
                              y_train.append(1)
                              counter += 1
                      if falsecounter < 30:
                          if iou < 0.3: ## if iou is less than 30% --> we have not identified an airplane
                              timage = imout[y:y+h,x:x+w]
                              resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                              X_train.append(resized)
                              y_train.append(0)
                              falsecounter += 1

  return np.array(X_train), np.array(y_train)

In [4]:
X_train, y_train = create_training_set(image_dir='Images', label_dir='Airplanes_Annotations')

PROCESSING IMAGES: 100%|██████████| 733/733 [08:23<00:00,  1.46IMAGE/s]


In [6]:
## RAM USAGE VERY HIGH ##
from sklearn.model_selection import train_test_split

ohy = tf.keras.utils.to_categorical(y_train, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X_train, ohy, test_size=0.1)

### LOAD FROM DRIVE OPTION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

In [4]:
X_test = np.load(r'/content/drive/MyDrive/Ashling/X_test.npy')
y_test = np.load(r'/content/drive/MyDrive/Ashling/y_test.npy')
y_train = np.load(r'/content/drive/MyDrive/Ashling/y_train.npy')
X_train = np.load(r'/content/drive/MyDrive/Ashling/X_train.npy')

### MODEL

In [10]:
vggmodel = VGG16(weights='imagenet', include_top=True)

for layers in (vggmodel.layers)[:15]:
    layers.trainable = False

X = vggmodel.layers[-2].output

predictions = Dense(2, activation="sigmoid")(X)

model_final = Model(inputs = vggmodel.input, outputs = predictions)

opt = Adam(learning_rate=0.0001)

model_final.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
tboard = TensorBoard(log_dir='./ieeercnn_vgg16_log',write_graph=True, write_images=False, write_grads=True)

hist = model_final.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, callbacks=[checkpoint, early, tboard])

Epoch 1/300
